# Day 7: Bridge Repair

The Historians take you to a familiar rope bridge over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole all the operators from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:
```
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
```
Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are always evaluated left-to-right, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: add (+) and multiply (*).

Only three of the above equations can be made true by inserting operators:

190: 10 19 has only one position that accepts an operator: between 10 and 19. Choosing + would give 29, but choosing * would give the test value (10 * 19 = 190).
3267: 81 40 27 has two positions for operators. Of the four possible configurations of the operators, two cause the right side to match the test value: 81 + 40 * 27 and 81 * 40 + 27 both equal 3267 (when evaluated left-to-right)!
292: 11 6 16 20 can be solved in exactly one way: 11 + 6 * 16 + 20.
The engineers just need the total calibration result, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is 3749.

Determine which equations could possibly be true. What is their total calibration result?

In [14]:
def get_data(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
        # lines = [line.replace(':', '') for line in lines]
        # lines = [line.split() for line in lines]
        # lines = [[int(x) for x in line] for line in lines]
    return lines

def get_equation(line):
    return line[0], line[1:]

# Naive Solution
Brute force solution to find all possible combinations of operators to calculate the test value. The number of locations where we can place an operator in a list of K numbers is K-1. We have two operators (+/*) so the total number of combinations is $2^{K-1}$. We can iterate over all possible combinations and check if the test value is equal to the calculated value.

In [7]:
def get_operators(n):
    # A generator for all possible combinations of operators for n numbers
    for i in range(2**(n-1)):
        operators = []
        for j in range(n-1):
            if i & (1<<j):
                operators.append('+')
            else:
                operators.append('*')
        yield operators

def find_equations(lines):
    result = 0
    for line in lines:
        test_value, numbers = get_equation(line)
        for operators in get_operators(len(numbers)):
            value = numbers[0]
            for i in range(len(operators)):
                if operators[i] == '+':
                    value += numbers[i+1]
                else:
                    value *= numbers[i+1]
            if value == test_value:
                result += test_value
                break
    return result

def run():
    lines = get_data('data.txt')
    result = find_equations(lines)
    print(result)

In [8]:
run()

28730327770375


# Optimized Solution
We can optimize the solution by using a recursive function to find all possible combinations of operators. We can pass the current value and the index of the next number to the function. The function will return the calculated value for all possible combinations of operators.

In [9]:
def find_equations_v2(lines):
    def find(test_value, numbers, value, index):
        # Edge case, if there's only one number, check if it is equal to the test value
        if len(numbers) == 1:
            print(f"Value: {value}, Test Value: {test_value} => {value == test_value}")
            return value == test_value

        if index == len(numbers):
            return value == test_value
        if find(test_value, numbers, value + numbers[index], index + 1):
            return True
        if find(test_value, numbers, value * numbers[index], index + 1):
            return True
        return False

    result = 0
    for line in lines:
        test_value, numbers = get_equation(line)
        if find(test_value, numbers, numbers[0], 1):
            result += test_value
    return result

def run():
    lines = get_data('data.txt')
    result = find_equations_v2(lines)
    print(result)

In [10]:
run()

28730327770375


# Part Two

The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a third type of operator.

The concatenation operator (||) combines the digits from its left and right inputs into a single number. For example, 12 || 345 would become 12345. All operators are still evaluated left-to-right.

Now, apart from the three equations that could be made true using only addition and multiplication, the above example has three more equations that can be made true by inserting operators:

- `156: 15 6` can be made true through a single concatenation: 15 || 6 = 156.
- `7290: 6 8 6 15` can be made true using 6 * 8 || 6 * 15.
- `192: 17 8 14` can be made true using 17 || 8 + 14.
Adding up all six test values (the three that could be made before using only + and * plus the new three that can now be made by also using ||) produces the new total calibration result of 11387.

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. What is their total calibration result?

In [15]:
def solve_part2(lines):
    total = 0
    for line in lines:
        line = line.strip()
        if not line:
            continue
        target_str, nums_str = line.split(':')
        target = int(target_str)
        nums = list(map(int, nums_str.split()))
        
        if can_make_target(nums, target):
            total += target
    return total


def can_make_target(nums, target):
    # If only one number, just check equality
    if len(nums) == 1:
        return nums[0] == target

    expressions = generate_expressions(nums)
    for expr in expressions:
        if evaluate_expression(expr) == target:
            return True
    return False


def generate_expressions(nums):
    """ Generate all expressions by inserting +, *, || between the numbers. 
        Return a list of lists, where each list is [num, op, num, op, num, ...].
        For convenience, we'll store the expression as a list alternating between numbers and operators, 
        starting and ending with numbers.
    """
    result = []
    ops = ['+', '*', '||']
    
    def backtrack(index, current_expr):
        # index refers to the position in nums
        # current_expr is a list like [num, op, num, op, ...]
        if index == len(nums) - 1:
            # Reached the last number
            result.append(current_expr[:])
            return
        
        # We are at nums[index], next gap is before nums[index+1]
        for op in ops:
            # Add operator and next number
            new_expr = current_expr + [op, nums[index+1]]
            backtrack(index+1, new_expr)
    
    # Start with the first number
    backtrack(0, [nums[0]])
    return result


def evaluate_expression(expr):
    """ Evaluate the expression list left-to-right. 
        expr is something like [num, '+', num, '||', num, '*', num]
        Evaluate strictly left to right:
        - Start with expr[0] as current value (must be a number)
        - For each operator+number pair:
          if op == '||', concatenate current_value and next_num as strings
          else if op == '+', do arithmetic addition
          else if op == '*', do arithmetic multiplication
    """
    # expr alternates between numbers and operators, starting and ending with a number.
    # So expr = [num, op, num, op, num, ...]
    value = expr[0]
    i = 1
    while i < len(expr):
        op = expr[i]
        next_num = expr[i+1]
        
        if op == '||':
            # Concatenate
            value = int(str(value) + str(next_num))
        elif op == '+':
            value = value + next_num
        elif op == '*':
            value = value * next_num
        i += 2
    
    return value


# Test with the given sample from the puzzle (should print 11387):
sample_lines = [
    "190: 10 19",
    "3267: 81 40 27",
    "83: 17 5",
    "156: 15 6",
    "7290: 6 8 6 15",
    "161011: 16 10 13",
    "192: 17 8 14",
    "21037: 9 7 18 13",
    "292: 11 6 16 20"
]

print(solve_part2(sample_lines))  # Expected 11387


11387


In [16]:
def run():
    lines = get_data('data.txt')
    result = solve_part2(lines)
    print(result)

In [17]:
run()

424977609625985
